In [9]:
import pandas as pd

In [11]:
df = pd.read_csv("../Data/twcs.csv")

In [12]:
df.head(5)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [5]:
df = df[df["inbound"] == True]

In [6]:
df.value_counts().sum()

211999

In [7]:
import re

In [8]:
# basic NLP task like getting data into NLP required format

In [9]:
import re

import re

def clean_text(text):
    text = text.lower()
    # remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # remove @mentions (including starting @)
    text = re.sub(r"@\w+", "", text)
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # remove emojis and special characters (keep only letters and space)
    text = re.sub(r"[^a-z\s]", "", text)
    # remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["text"].apply(clean_text)

In [10]:
# df = df[["tweet_id", "clean_text"]]

In [11]:
pd.set_option("display.max_rows", 50)

In [12]:
df[["clean_text"]].head(20)

,clean_text
1,and how do you propose we do that
2,i have sent several private messages and no on...
4,i did
6,is the worst customer service
8,you gonna magically change your connectivity f...
10,since i signed up with yousince day
12,yall lie about your great connection bars lte ...
14,whenever i contact customer support they tell ...
16,would you like me to email you a copy of one s...
18,i received this from your corporate office wou...


In [13]:
# to create a NLP model for chatbot we need to first create main two columns text and intent. 
# Intent need to create manually because data i noisy so follwoing is the manual way 

In [14]:
# df["label"] = ""
# df.loc[0, "label"] = "customer_service_complaint"
# df.loc[1, "label"] = "general_query"
# df.loc[2, "label"] = "connectivity_issue"

In [15]:
# df = df[df["label"] != ""]
# df = df[df["clean_text"].str.split().str.len() > 2]
# removing bad rows

In [16]:
# df[["clean_text", "label"]].to_csv(
#     "labeled_tweets.csv",
#     index=False
# )
# save labeled data 

In [ ]:
# Now Using cleaned csv convert model into pkl

In [18]:
df = pd.read_csv("../Data/labeled_tweets.csv")

In [20]:
df.rename(columns={'clean_text': 'cleaned_text'}, inplace=True)

In [22]:
df["cleaned_text"].isna().sum()

10156

In [24]:
df = df.dropna(subset=["cleaned_text", "label"])

In [26]:
df["cleaned_text"].isna().sum()

0

In [28]:
X = df["cleaned_text"]
y = df["label"]

In [30]:
X

0                         and how do you propose we do that
1         i have sent several private messages and no on...
2                                                     i did
3                             is the worst customer service
4         you gonna magically change your connectivity f...
                                ...                        
574746    hey apple thanks for not warning me that the n...
574747    why do photos take up so much space on my phon...
574748             i need help with my icloud photo library
574749                                   when does you good
574751    watford bought crayfish tails the other day wh...
Name: cleaned_text, Length: 564596, dtype: object

In [32]:
y

0                      general_query
1                      account_issue
2                      general_query
3         customer_service_complaint
4                 connectivity_issue
                     ...            
574746    customer_service_complaint
574747                 general_query
574748                 general_query
574749                 general_query
574751                 general_query
Name: label, Length: 564596, dtype: object

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

model = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
        stop_words="english"
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"
    ))
])

model.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2),
                                 stop_words='english')),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [37]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                            precision    recall  f1-score   support

             account_issue       0.97      0.94      0.95      6077
             billing_issue       0.83      0.89      0.86      5378
        connectivity_issue       0.93      0.90      0.92      4957
customer_service_complaint       0.67      0.85      0.75      3129
             general_query       0.98      0.97      0.97     81897
         positive_feedback       0.98      0.97      0.98     11482

                  accuracy                           0.96    112920
                 macro avg       0.89      0.92      0.90    112920
              weighted avg       0.96      0.96      0.96    112920



In [40]:
import joblib
joblib.dump(model, "intent_model.pkl")

['intent_model.pkl']